# Preprocess c file

TCI percentiles per city-country

`Last update`: 2021-04-15

In [123]:
import subprocess
import pandas as pd

def _create_cs_file(path_c):
    
    df = pd.read_csv(path_c) \
        .drop_duplicates('region_slug') \
        .dropna()

    for k in df.columns:
        if k != 'region_slug':
            df[k] = pd.to_numeric(df[k])
        if k == 'region_slug':
            df[k] = df[k].astype(str) 

    df_c = df \
        .assign(c_low_p01 = lambda x: (x['q25']-x['q01'])/(x['q75']-x['q25']),
                c_upp_p99 = lambda x: (x['q99']-x['q75'])/(x['q75']-x['q25']),
                c_low_p05 = lambda x: (x['q25']-x['q05'])/(x['q75']-x['q25']),
                c_upp_p95 = lambda x: (x['q95']-x['q75'])/(x['q75']-x['q25'])
               ) \
        [['region_slug', 'c_low_p01', 'c_upp_p99', 'c_low_p05', 'c_upp_p95']]
    
    df_c['c_min'] = df_c[['c_low_p01', 'c_upp_p99', 'c_low_p05', 'c_upp_p95']].min(axis=1)
    
    return df_c

def _save_cs_file(): 
    
    df_c = _create_cs_file(path_c = '~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv')
    print(df_c.shape)
    print(df_c.describe())
    df_c.to_csv('~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_c_iqr.csv', index=False)
    

## Copy percentile

In [114]:
!cp ~/shared/spd-sdv-omitnik-waze/waze_tci/region_quantiles/quantiles/regions_quantiles.csv ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/raw/regions_quantiles.csv

!sed "s/\[//g;s/\]//g;s/['\"]//g" ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/raw/regions_quantiles.csv > ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr_00.csv

In [115]:
%%bash 

echo -e "region_slug,mean,var_p, var_s,q10,q20,q30,q40,q50,q60,q70,q80,q90,q01,q99,q25,q75,q05,q95" | cat - ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr_00.csv > ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv

In [116]:
!rm ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr_00.csv

In [117]:
!head -2 ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv

region_slug,mean,var_p, var_s,q10,q20,q30,q40,q50,q60,q70,q80,q90,q01,q99,q25,q75,q05,q95
aguascalientes,8039136.868743886,8862843413565.455,8887326406420.607,3829261.1993683483,5605996.229991711,6600013.058713919,6932325.881132038,7473441.530126513,8626943.901942156,9718372.086622398,11136396.36923769,11136396.36923769,2151065.466800198,14393234.680959623,6215087.576892198,10248917.983242217,3349729.7486610203,12964231.367048915


In [118]:
!wc -l ~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv

46 /home/soniame/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_iqr.csv


## Save c's file

In [124]:
_save_cs_file()

(45, 6)
       c_low_p01  c_upp_p99  c_low_p05  c_upp_p95      c_min
count  45.000000  45.000000  45.000000  45.000000  45.000000
mean    0.824890   1.399746   0.602593   0.745087   0.511502
std     0.420575   0.672825   0.338406   0.314838   0.238458
min     0.213163   0.553016   0.165875   0.308937   0.165875
25%     0.510229   0.880794   0.353048   0.508614   0.353048
50%     0.663578   1.225445   0.494875   0.640405   0.457901
75%     1.086996   1.687227   0.773777   0.922599   0.620649
max     1.766687   3.101491   1.580424   1.567010   1.482831


In [125]:
df = pd.read_csv('~/shared/spd-sdv-omitnik-waze/corona/cleaning/data/staging/cities_c_iqr.csv')

In [126]:
df.head()

,region_slug,c_low_p01,c_upp_p99,c_low_p05,c_upp_p95,c_min
0,aguascalientes,1.007485,1.027390,0.710332,0.673135,0.673135
1,arequipa,1.370780,1.318878,0.967781,0.620649,0.620649
2,asuncion,0.615222,1.589508,0.503198,0.604833,0.503198
3,barcelona,0.564193,1.984999,0.447251,1.037351,0.447251
4,barquisimeto,0.590701,1.705025,0.494875,0.761201,0.494875
